In [1]:
!pip install requests beautifulsoup4
from urllib3.poolmanager import key_fn_by_scheme
!pip install selenium
!pip install pymysql

In [ ]:
import time
import json
import requests
import pymysql
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

DB_CONFIG = {
    "host": "192.168.76.xxx",
    "user": "wohhahha",
    "password": "wohhahha",
    "db": "motor_chata",
    "charset": "utf8mb4",
}

LIST_URL = "https://www.car.go.kr/ri/stat/list.do"
DETAIL_URL = "https://www.car.go.kr/ri/stat/detail.do"

TARGET_BRANDS = [
    "기아", "현대", "제네시스",
    "케이지모빌리티", "케이지", "KG",
    "르노", "르노코리아",
    "벤츠",
    "비엠더블유", "BMW",
    "테슬라",
    "볼보",
    "BYD", "BYD코리아", "비와이디",
]

LAST_PAGE = 864


In [ ]:
conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor()

insert_sql = """
INSERT INTO recall_info (
    page_no, maker, title, recall_id, ctype,
    maker_detail, car_name, 생산기간, 시정기간,
    대상수량, 장치분류, 결함내용, 시정방법, 기타문의
)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""


print("DB 연결 완료")


In [ ]:

# Selenium 드라이버
driver = webdriver.Chrome()
driver.implicitly_wait(5)
driver.get(LIST_URL)

# detail.do용 requests 세션
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Referer": LIST_URL,
    "Origin": "https://www.car.go.kr",
})

# 쿠키 맞추려고 한 번 GET
session.get(LIST_URL)

print("Selenium & requests 준비 완료")


In [ ]:

def parse_maker_title(title_full: str):
    """[제작사] 제목 → (maker, title) 분리"""
    maker = ""
    title = title_full.strip()
    if "[" in title_full and "]" in title_full:
        maker = title_full.split("]")[0].replace("[", "").strip()
        title = title_full.split("]")[1].strip()
    return maker, title


def parse_onclick(onclick_value: str):
    """
    $main.event.detailView('5869','O'); return false;
    이런 문자열에서 recall_id, ctype 추출
    """
    if "detailView" not in onclick_value:
        return None, None

    inner = onclick_value.split("detailView(")[1].split(")")[0]
    parts = [p.strip().strip("'") for p in inner.split(",")]
    if len(parts) >= 2:
        return parts[0], parts[1]
    return None, None


def fetch_detail_html(session: requests.Session, recall_id: str, ctype: str = "O") -> str:
    """detail.do 에 POST로 요청해서 HTML 전체를 문자열로 반환"""
    payload = {
        "currentPageNo": "",
        "ctype": ctype,
        "searchOriginalMakerName": "",
        "searchOriginalMakerCode": "",
        "searchProductName": "",
        "searchFromDate": "",
        "searchToDate": "",
        "recallId": recall_id,
    }

    resp = session.post(DETAIL_URL, data=payload)
    if resp.status_code != 200:
        print(f"⚠ detail.do 호출 실패 {resp.status_code} / recallId={recall_id}")
        return ""

    return resp.text


def parse_detail_table(html: str) -> dict:
    """상세페이지의 표 → dict 형태로 파싱"""
    soup = BeautifulSoup(html, "html.parser")
    result = {}

    table = soup.find("table", class_="table-stat")
    if not table:
        return result

    tbody = table.find("tbody")
    if not tbody:
        return result

    for tr in tbody.find_all("tr"):
        th_tags = tr.find_all("th")
        td_tags = tr.find_all("td")

        for th, td in zip(th_tags, td_tags):
            key = th.get_text(strip=True)
            val = td.get_text("\n", strip=True)
            if key:
                result[key] = val

    return result

In [ ]:
total_saved = 0

for page in range(1, LAST_PAGE + 1):
    print(f"\n=== {page} 페이지 ===")

    if page != 1:
        driver.execute_script(f"$main.event.fn_search({page});")
        time.sleep(1)

    items = driver.find_elements(By.CSS_SELECTOR, "ul.board-hrznt-list > li")
    print("리콜 건수:", len(items))

    for li_tag in items:
        a_tag = li_tag.find_element(By.CSS_SELECTOR, "a")
        strong_tag = a_tag.find_element(By.CSS_SELECTOR, "strong")

        title_full = strong_tag.text.strip()
        maker, title = parse_maker_title(title_full)

        # 브랜드 필터링
        if not any(keyword in maker for keyword in TARGET_BRANDS):
            continue

        onclick_val = a_tag.get_attribute("onclick") or ""
        recall_id, ctype = parse_onclick(onclick_val)

        if not recall_id:
            print("⚠ recall_id 추출 실패")
            continue

        # 상세 HTML 가져오기
        detail_html = fetch_detail_html(session, recall_id, ctype)
        detail_info = parse_detail_table(detail_html)

        # 🔥 INSERT — 컬럼만 저장 (14개)
        cursor.execute(
            insert_sql,
            (
                page,                  # page_no
                maker,                 # maker
                title,                 # title
                recall_id,             # recall_id
                ctype,                 # ctype
                detail_info.get("제작(수입)사"),
                detail_info.get("차명"),
                detail_info.get("생산기간"),
                detail_info.get("시정기간"),
                detail_info.get("대상수량"),
                detail_info.get("장치분류"),
                detail_info.get("결함내용"),
                detail_info.get("시정방법"),
                detail_info.get("기타문의"),
            )
        )

        total_saved += 1
        print(f"[저장] p{page} | {maker} | {title} | id={recall_id}")
        time.sleep(0.2)

conn.commit()
cursor.close()
conn.close()
driver.quit()

print("\n크롤링 완료")
print("총 저장 건수:", total_saved)


In [ ]:
import pymysql
import pandas as pd

conn = pymysql.connect(**DB_CONFIG)
cur = conn.cursor()

cur.execute("SELECT * FROM recall_info")
rows = cur.fetchall()

# 컬럼명 불러오기
cols = [desc[0] for desc in cur.description]

# DataFrame 만들기
df = pd.DataFrame(rows, columns=cols)

cur.close()
conn.close()

# 🔥 여기 바로 아래에 CSV 저장
df.to_csv("recall.csv", index=False, encoding="utf-8-sig")
print("CSV 저장 완료")
